In [1]:
#Set-up
import os
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import seaborn as sns
import warnings
from IPython.display import clear_output
import nibabel as nib

import torch
import torchvision.models as models
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn as nn
from torch.optim import lr_scheduler
import torch.nn.functional as F
import torch.utils.data as utils
import time
import copy

In [24]:
x = torch.load("../../../data/data.pt")
y = torch.load("../../../data/labels_14.pt")
n = x.shape[0]
y = y.to(dtype=torch.long)
print(x.shape,y.shape)

torch.Size([536, 3, 224, 224]) torch.Size([536])


In [3]:
fcp_x = torch.load("../../../data/fcp_data.pt")
fcp_y = torch.load("../../../data/fcp_labels_14.pt")
print(fcp_x.shape,fcp_y.shape)
fcp_n = fcp_x.shape[0]

torch.Size([1034, 3, 224, 224]) torch.Size([1034])


In [59]:
combined_x = torch.cat((x,fcp_x))
combined_y = torch.cat((y,fcp_y))
# combined_y = torch.reshape(combined_y,(fcp_n + n,1))
N = combined_x.shape[0]
print(combined_x.shape,combined_y.shape)

torch.Size([1570, 3, 224, 224]) torch.Size([1570])


In [60]:
x_normalized = torch.Tensor(combined_x.shape)
for i in range(N):
    x_normalized[i] = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225]).__call__(combined_x[i,:,:])

In [61]:
USE_GPU = True

dtype = torch.float32 # we will be using float throughout this tutorial

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print('using device:', device)

using device: cuda


In [62]:
# dataset = utils.TensorDataset(combined_x, combined_y)

In [70]:
train_indices = [i for i in range(N) if (i % 5 == 0 or i % 5 == 2 or i % 5 == 4)]
val_indices = [i for i in range(N) if i % 5 == 3]
test_indices = [i for i in range(N) if i not in train_indices and i not in val_indices]

print("There are %d (%.2f) training data points, %d (%.2f) validation data points,and %d (%.2f) test data points" %(len(train_indices),len(train_indices)/N,len(val_indices),\
                                                                                                                len(val_indices)/N,len(test_indices),len(test_indices)/N))
train_dataset = utils.TensorDataset(x_normalized[:],combined_y[:])
val_dataset = utils.TensorDataset(x_normalized[val_indices],combined_y[val_indices])      
dataloaders = {}
dataloaders['train'] = torch.utils.data.DataLoader(train_dataset, batch_size=4,
                                             shuffle=True,num_workers=4)
dataloaders['val'] = torch.utils.data.DataLoader(train_dataset, batch_size=4,
                                             shuffle=True,num_workers=4)

There are 942 (0.60) training data points, 314 (0.20) validation data points,and 314 (0.20) test data points


In [72]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    train_loss =[]
    val_loss =[]
    train_acc_history = []
    val_acc_history = []

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            
            if phase == 'train':
                epoch_loss = running_loss / (N)
                epoch_acc = running_corrects.double() / (N )
            else:
                epoch_loss = running_loss / (N )
                epoch_acc = running_corrects.double() / (N)
            
            if phase == 'train':
                train_acc_history.append(epoch_acc)
                train_loss.append(epoch_loss)
            else:
                val_acc_history.append(epoch_acc)
                val_loss.append(epoch_loss)
            
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))
    
    plt.subplot(2, 1, 1)
    plt.plot(train_loss, '-o')
    plt.plot(val_loss, '-o')
    plt.legend(['train', 'val'], loc='upper left')
    plt.xlabel('epoch')
    plt.ylabel('loss')

    plt.subplot(2, 1, 2)
    plt.plot(train_acc_history, '-o')
    plt.plot(val_acc_history, '-o')
    plt.legend(['train', 'val'], loc='upper left')
    plt.xlabel('epoch')
    plt.ylabel('accuracy')
    plt.show()

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [73]:
model_ft = models.resnet50(pretrained=True)

num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 14)

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
# optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)
optimizer_ft = optim.Adam(model_ft.parameters(), lr=0.0001)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=5, gamma=0.1)

In [74]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=20)

Epoch 0/19
----------
train Loss: 1.9759 Acc: 0.3516
val Loss: 2.6893 Acc: 0.0631

Epoch 1/19
----------
train Loss: 1.8893 Acc: 0.3465
val Loss: 2.4825 Acc: 0.0631

Epoch 2/19
----------


KeyboardInterrupt: 

In [17]:
# torch.save(model_ft.state_dict(), "best_classification_model.pth")
def check_accuracy(data,labels,model):
    print('Checking accuracy on test set')   
    num_correct = 0
    num_samples = 0
    model.eval()  # set model to evaluation mode
    with torch.no_grad():
        data = data.to(device=device, dtype=dtype)  # move to device, e.g. GPU
        labels = labels.to(device=device, dtype=torch.long)
        scores = model(data)
        _, preds = scores.max(1)
        num_correct += (preds == labels).sum()
        num_samples += preds.size(0)
        acc = float(num_correct) / num_samples
        print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))

In [20]:
check_accuracy(x_normalized[test_indices], combined_y[test_indices],model_ft)

Checking accuracy on test set
Got 101 / 314 correct (32.17)
